## Introduction 

A critical skill for deploying (and even testing) complex models with pre-processing.

Pipelines are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

Many data scientists hack together models without pipelines, but pipelines have some important benefits. Those include:

- Cleaner Code: Accounting for data at each step of preprocessing can get messy. With a pipeline, you won't need to manually keep track of your training and validation data at each step.
- Fewer Bugs: There are fewer opportunities to misapply a step or forget a preprocessing step.
- Easier to Productionize: It can be surprisingly hard to transition a model from a prototype to something deployable at scale. We won't go into the many related concerns here, but pipelines can help.
- More Options for Model Validation: You will see an example in the next tutorial, which covers cross-validation.

A machine learning pipeline is a way to codify and automate the workflow it takes to produce a machine learning model. Machine learning pipelines consist of multiple sequential steps that do everything from data extraction and preprocessing to model training and deployment.

<img src="https://miro.medium.com/max/1838/1*13uV1HSrv5NefuFo85lkrg.png"/>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
melb = pd.read_csv('dataset/melb_data.csv')

In [3]:
y_target = melb['Price']

In [4]:
x_features = melb.drop(['Price'], axis=1)

In [5]:
# Divide data into training and validation subsets
x_train, x_val, y_train, y_val = train_test_split(x_features, y_target, test_size=0.2, random_state=0)

In [6]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
cols_categorical = [col for col in x_train.columns if x_train[col].dtype == 'object' and x_train[col].nunique() < 10]
cols_categorical

['Type', 'Method', 'Regionname']

In [7]:
# Select numerical columns
cols_numericals = x_train.select_dtypes(exclude=['object']).columns
cols_numericals

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')

In [8]:
list(cols_numericals)

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [9]:
# the way kaggle does
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

In [12]:
# Keep selected columns only
x_train = x_train[(cols_categorical + list(cols_numericals))].copy()
x_val = x_val[(cols_categorical + list(cols_numericals))].copy()

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10864 entries, 12167 to 2732
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           10864 non-null  object 
 1   Method         10864 non-null  object 
 2   Regionname     10864 non-null  object 
 3   Rooms          10864 non-null  int64  
 4   Distance       10864 non-null  float64
 5   Postcode       10864 non-null  float64
 6   Bedroom2       10864 non-null  float64
 7   Bathroom       10864 non-null  float64
 8   Car            10815 non-null  float64
 9   Landsize       10864 non-null  float64
 10  BuildingArea   5708 non-null   float64
 11  YearBuilt      6557 non-null   float64
 12  Lattitude      10864 non-null  float64
 13  Longtitude     10864 non-null  float64
 14  Propertycount  10864 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 1.3+ MB


In [13]:
x_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


We take a peek at the training data with the head() method below. Notice that the data contains both categorical data and columns with missing values. With a pipeline, it's easy to deal with both!
We construct the full pipeline in three steps.

## Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:

- imputes missing values in numerical data, and
- imputes missing values and applies a one-hot encoding to categorical data.

Use columnTransform to aplly different preprocessing to different columns:
- select from DataFrame columns by name
- passthrough or drop unspecified columns

Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

SimpleImputer?

If "constant", then replace missing values with fill_value. Can be
     - used with strings or numeric data.
      
If left to the default, fill_value will be 0 when imputing numerical
    - data and "missing_value" for strings or object data types.

In [18]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification.
Bellow, we create a pipeline to perform simple imputer using most_frequent strategy and after that, we use onehot enconder to perform the categorical transformation.

steps : list of tuple
- List of (name, transform) tuples (implementing fit/transform) that are chained, in the order in which they are chained, with the last object an estimator.

In [19]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # first that
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # then, that
])

transformers : list of tuples
- List of (name, transformer, columns) tuples specifying the transformer objects to be applied to subsets of the data.

In [20]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, cols_numericals),
        ('cat', categorical_transformer, cols_categorical)
    ])

## Step 2: Define the Model

Next, we define a random forest model with the familiar RandomForestRegressor class.

In [22]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

Finally, we use the Pipeline class to define a pipeline that bundles the preprocessing and modeling steps. There are a few important things to notice:

- With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)
- With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)

In [23]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(x_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(x_val)

# Evaluate the model
score = mean_absolute_error(y_val, preds)
print('MAE:', score)

MAE: 160679.18917034855


## Conclusion
Pipelines are valuable for cleaning up machine learning code and avoiding errors, and are especially useful for workflows with sophisticated data preprocessing.

# Exercise

In [26]:
# do everything again for practice
import pandas as pd

data_train = pd.read_csv('dataset/home_data_train.csv', index_col='Id')

data_test = pd.read_csv('dataset/home_data_test.csv', index_col='Id')

data_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# Remove rows with missing target, separate target from predictors
data_train.dropna(subset=['SalePrice'], inplace=True, axis=0)
y = data_train['SalePrice'].copy()
x_features = data_train.drop(['SalePrice'], axis=1).copy()

In [29]:
x_features.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [30]:
y

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

In [31]:
# Break off validation set from training data
x_train, x_valid, y_train, y_valid = train_test_split(x_features, y, random_state=0, test_size=0.2)

In [36]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols_low_cardinality = [col for col in x_train.columns if x_train[col].dtype == 'object' 
                                                                  and x_train[col].nunique() < 10]
categorical_cols_low_cardinality

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [39]:
# Select numerical columns
numerical_cols = list(x_train.select_dtypes(include=['int64', 'float64']).columns)
numerical_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [47]:
# Keep selected columns only
selected_cols = categorical_cols_low_cardinality + numerical_cols
x_train = x_train[selected_cols].copy()
x_valid = x_valid[selected_cols].copy()
data_test = data_test[selected_cols].copy()

In [48]:
x_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


In [52]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [53]:
# Preprocessing for numerical data
numerical_preprocessing = SimpleImputer(strategy='constant')

In [57]:
# Preprocessing for categorical data
categorical_preprocessing = Pipeline(steps=[
    ('imputeMissingValues', SimpleImputer(strategy='most_frequent')), #first step
    ('OneHotEnconding', OneHotEncoder(handle_unknown='ignore')) #second step
])

In [58]:
# Bundle preprocessing for numerical and categorical data
# using collumn transform for this
preprocessing = ColumnTransformer(transformers=[('numerical', numerical_preprocessing, numerical_cols),
                                                ('categorical', categorical_preprocessing, categorical_cols_low_cardinality)])

In [61]:
# Define model
rf = RandomForestRegressor(n_estimators=100, random_state=0)

In [62]:
# Bundle preprocessing and modeling code in a pipeline
model_pipeline = Pipeline(steps=[('preprocessing', preprocessing), 
                                 ('model', rf)])

pipeline.fit = Fit all the transforms one after the other and transform the
data, then fit the transformed data using the final estimator.
pipeline.fit_transform =

In [66]:
# Preprocessing of training data, fit model 
model_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath',...
                                                  

In [67]:
# Preprocessing of validation data, get predictions
pred = model_pipeline.predict(x_valid)

In [69]:
print('MAE:', mean_absolute_error(y_valid, pred))

MAE: 17861.780102739725


The code yields a value around 17862 for the mean absolute error (MAE). In the next step, you will amend the code to do better.


# Step 1: Improve the performance

### Part A

Now, it's your turn!  In the code cell below, define your own preprocessing steps and random forest model.  Fill in values for the following variables:
- `numerical_transformer`
- `categorical_transformer`
- `model`

To pass this part of the exercise, you need only define valid preprocessing steps and a random forest model.

In [72]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('inpute', SimpleImputer(strategy='most_frequent')), 
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols_low_cardinality)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

### Part B

Run the code cell below without changes.

To pass this step, you need to have defined a pipeline in **Part A** that achieves lower MAE than the code above.  You're encouraged to take your time here and try out many different approaches, to see how low you can get the MAE!  (_If your code does not pass, please amend the preprocessing steps and model in Part A._)

In [73]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

In [76]:
# Preprocessing of training data, fit model 
my_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath...
                                                 

In [78]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(x_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 17553.371061643833


# Step 2: Generate test predictions

Now, you'll use your trained model to generate predictions with the test data.

In [79]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(data_test)